# ETL da Fato de Vendas

Este notebook constrói a tabela fato de vendas a partir dos dados
transacionais da Olist, utilizando as dimensões criadas na camada Silver
e gravando o resultado final na camada Gold.

In [0]:
# Importações
from pyspark.sql.functions import col, to_timestamp

In [0]:
# Paths Bronze
BRONZE_PATH = "dbfs:/Workspace/dados_mpv/"

In [0]:
# Leitura dos dados transacionais (Bronze)
orders_df = spark.read.option("header", "true").csv(
    f"{BRONZE_PATH}olist_orders_dataset.csv"
)

items_df = spark.read.option("header", "true").csv(
    f"{BRONZE_PATH}olist_order_items_dataset.csv"
)

In [0]:
# Filtro de pedidos válidos
orders_df = orders_df.filter(col("order_status") == "delivered")

In [0]:
# Join Orders + Itens
fato_base = orders_df.join(
    items_df,
    orders_df.order_id == items_df.order_id,
    "inner"
)

In [0]:
# Construção da Fato de Vendas
fato_vendas = fato_base.select(
    orders_df.order_id.alias("id_pedido"),
    orders_df.customer_id.alias("id_cliente"),
    items_df.product_id.alias("id_produto"),
    items_df.seller_id.alias("id_vendedor"),
    orders_df.order_purchase_timestamp.alias("id_data"),
    items_df.price.cast("double").alias("valor_item"),
    items_df.freight_value.cast("double").alias("valor_frete"),
    items_df.order_item_id.cast("int").alias("quantidade_itens")
)

In [0]:
# Escrita na camada Gold
fato_vendas.write.mode("overwrite").saveAsTable(
    "workspace.gold.fato_vendas"
)

In [0]:
# Validação
spark.sql("SELECT * FROM workspace.gold.fato_vendas LIMIT 10").show()

+--------------------+--------------------+--------------------+--------------------+-------------------+----------+-----------+----------------+
|           id_pedido|          id_cliente|          id_produto|         id_vendedor|            id_data|valor_item|valor_frete|quantidade_itens|
+--------------------+--------------------+--------------------+--------------------+-------------------+----------+-----------+----------------+
|00010242fe8c5a6d1...|3ce436f183e68e078...|4244733e06e7ecb49...|48436dade18ac8b2b...|2017-09-13 08:59:02|      58.9|      13.29|               1|
|00018f77f2f0320c5...|f6dd3ec061db4e398...|e5f2d52b802189ee6...|dd7ddc04e1b6c2c61...|2017-04-26 10:53:06|     239.9|      19.93|               1|
|000229ec398224ef6...|6489ae5e4333f3693...|c777355d18b72b67a...|5b51032eddd242adc...|2018-01-14 14:33:31|     199.0|      17.87|               1|
|00024acbcdf0a6daa...|d4eb9395c8c0431ee...|7634da152a4610f15...|9d7a1d34a50524090...|2018-08-08 10:00:35|     12.99|      12